In [38]:
import  pandas as pd
import numpy as np
import selenium

In [39]:
df = pd.read_csv('df_27.csv', sep=',')
df

,Unnamed: 0.1,Unnamed: 0,Event ID,Event Date,Event Text,Story ID,City,District,Province,Latitude,Longitude,Year
0,275076,1199488,37533394,2023-01-02,"Abduct, hijack, or take hostage",56609779,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2023
1,275077,1199489,37533336,2023-01-02,Make statement,56609868,Voronezh,NaN,Voronezhskaya Oblast',51.6720,39.1843,2023
2,275078,1199491,37533337,2023-01-02,fight with small arms and light weapons,56609902,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2023
3,275079,1199492,37533164,2023-01-02,fight with small arms and light weapons,56609972,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2023
4,275080,1199496,37534493,2023-01-02,Use conventional military force,56610355,Bryansk,Bryanskiy Rayon,Bryanskaya Oblast',53.2521,34.3717,2023
5,275081,1199531,37531460,2023-01-03,Praise or endorse,56605975,St. Petersburg,NaN,Sankt-Peterburg,59.9386,30.3141,2023
6,275082,1199532,37534529,2023-01-03,Demonstrate or rally,56606014,Samara,NaN,Samarskaya Oblast',53.2001,50.1500,2023
7,275083,1199533,37534530,2023-01-03,Demonstrate or rally,56606014,Tol'yatti,NaN,Samarskaya Oblast',53.5303,49.3461,2023
8,275084,1199545,37531585,2023-01-03,Use conventional military force,56606380,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2023
9,275085,1199547,37531914,2023-01-03,Make statement,56606530,Samara,NaN,Samarskaya Oblast',53.2001,50.1500,2023


In [40]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [41]:
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome()


In [42]:
# driver = webdriver.Chrome()
options = webdriver.ChromeOptions()
# options.add_argument("--headless")
# options.page_load_strategy = 'eager'
driver.get("https://yandex.ru/maps/")



In [43]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from fake_useragent import UserAgent
# ua = UserAgent()


In [44]:
def get_municipality(latitude, longitude):
    try:

        # Находим поле поиска

        search_box = driver.find_element(By.CSS_SELECTOR, "input.input__control")
        search_box.clear()

        # Вводим координаты
        coordinates = f"{latitude}, {longitude}"
        search_box.send_keys(coordinates)
        print(Keys.RETURN)
        search_box.send_keys(Keys.RETURN)
        time.sleep(0.1)  # Ждём загрузки результата


        # Извлекаем информацию о муниципалитете
        # Считываем первый результат в правом блоке
        search_box = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.toponym-card-title-view__description'))
            )

        municipality = search_box.text
        print(municipality)
        search_box = driver.find_element(By.XPATH, "//button[contains(@aria-label, 'Закрыть')]") 
        search_box.click()

        return municipality
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

In [45]:
# from tqdm import tqdm

# Создаём столбец для результатов
df["municipality"] = None
# Парсим каждую координату
for idx, row in df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    municipality = get_municipality(latitude, longitude)
    df.at[idx, "municipality"] = municipality
    print(f"{idx}/{10500}")
    if (idx%2500==0 and idx!=0) or (idx%4100==0 and idx!=0):
        time.sleep(60*3)

    

# Сохраняем результаты в файл
df.head()


Гайский муниципальный округ, Оренбургская область
0/10500

улица Бакунина, 47, Воронеж, 394030
1/10500

Гайский муниципальный округ, Оренбургская область
2/10500

Гайский муниципальный округ, Оренбургская область
3/10500

Арсенальская улица, Брянск
4/10500

Дворцовая площадь, Санкт-Петербург
5/10500

Чернореченская улица, 67, Самара, 443013
6/10500

квартал 14А, Тольятти, Самарская область
7/10500

Гайский муниципальный округ, Оренбургская область
8/10500

Чернореченская улица, 67, Самара, 443013
9/10500

село Сумы, Трескинский сельсовет, Колышлейский район, Пензенская область
10/10500

Гайский муниципальный округ, Оренбургская область
11/10500

Чайковское сельское поселение, Нытвенский городской округ, Пермский край
12/10500

Чайковское сельское поселение, Нытвенский городской округ, Пермский край
13/10500

Арылахский наслег, Верхнеколымский улус (район), Республика Саха (Якутия)
14/10500

Арылахский наслег, Верхнеколымский улус (район), Республика Саха (Якутия)
15/10

,Unnamed: 0.1,Unnamed: 0,Event ID,Event Date,Event Text,Story ID,City,District,Province,Latitude,Longitude,Year,municipality
0,275076,1199488,37533394,2023-01-02,"Abduct, hijack, or take hostage",56609779,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2023,"Гайский муниципальный округ, Оренбургская область"
1,275077,1199489,37533336,2023-01-02,Make statement,56609868,Voronezh,NaN,Voronezhskaya Oblast',51.6720,39.1843,2023,"улица Бакунина, 47, Воронеж, 394030"
2,275078,1199491,37533337,2023-01-02,fight with small arms and light weapons,56609902,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2023,"Гайский муниципальный округ, Оренбургская область"
3,275079,1199492,37533164,2023-01-02,fight with small arms and light weapons,56609972,Kherson,NaN,Orenburgskaya Oblast',51.4000,58.6167,2023,"Гайский муниципальный округ, Оренбургская область"
4,275080,1199496,37534493,2023-01-02,Use conventional military force,56610355,Bryansk,Bryanskiy Rayon,Bryanskaya Oblast',53.2521,34.3717,2023,"Арсенальская улица, Брянск"


In [46]:
df.to_csv('all_fin28.csv')